In [1]:
from transformer_model import Transformer # this is the transformer.py file
import torch
import numpy as np
from datasets import load_dataset

dataset = load_dataset("findnitai/english-to-hinglish")

c:\Users\AAHIL ALWANI\Desktop\FYP\Web App\Meme_AI\meme\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 189102
    })
})

In [3]:
# Access the train split
train_dataset = dataset["train"]

# Get the values of the 'translation' column
translations = train_dataset["translation"]

english_sentences=[]
urdu_sentences=[]
# Print the first few values
for translation in translations:
    english_sentences.append(translation['en'].lower())
    urdu_sentences.append(translation['hi_ng'].lower())

In [4]:
#separate english and urdu urdu
english_sentences

["what's the name of the movie",
 'hi, the rotten tomatoes score is great but the meta critic score seems a little low a movie of this quality. ',
 'do you think you will like the movie',
 'what kind of movie is it',
 'when was the movie made?',
 'wonder woman, i think i would enjoy this movie very much',
 'whats the name of the movie',
 'it is a action movie set in the dc comic world',
 'who stars in the movie',
 'the movie was made in 2015',
 'did it do well in theaters?',
 'did i have to see other movies to get it?',
 'the main character is gal gadot, but also stars  chris pine, robin wright and danny huston',
 'what is the movie about?',
 'no i think this movie is a stand alone piece that you wont need knoledge of other movies to understand it',
 'it is a war movie action movie',
 'do you think this is a family movie for the kids',
 'are you familiar with any of the actors in the movie?',
 'some of them',
 'i think its acceptable for a family to watch with children',
 'even though 

In [5]:
urdu_sentences

['film ka kya naam hai',
 'namaste, sada hua tomatoes score mahaan hai, lekin meta critic score is gunavatta kee philm se thoda kam lagata hai.',
 'kya aapako lagata hai ki aapako film pasand aaegee',
 'yah kis tarah kee philm hai',
 'film  kab banee thee?',
 'aashchary hai ki mahila, mujhe lagata hai ki mujhe is film mein bahut maja aaega',
 'film ka kya naam hai',
 'yah deesee komik duniya mein sthaapit ek ekshan philm hai',
 ': film mein kaun sitaare hain',
 'movie 2015 mein banee thee',
 'kya isane theaters mein achchha pradarshan kiya?',
 'kya mujhe ise paane kelie any movie dekhanee hongee?',
 'main character gal gadot hai, lekin chris pine, robin wright aur danny huston bhi hain',
 'movie ke baare mein kya hai?',
 'nahin, mujhe lagata hai ki yah movie ek stand alone piece hai jise samajhane ke lie aapako movies  ke knowledge kee aavashyakata nahin hogee',
 'yah ek war moovie ekshan movie hai',
 'kya aapako lagata hai ki yah bachchon kelie ek paarivaarik movie hai',
 'kya aap mov

In [6]:
START_TOKEN = ''
PADDING_TOKEN = ''
END_TOKEN = ''

## Insert the data into CSV

In [7]:
# import csv
# import re

# # Function to remove URLs from text
# def remove_urls(text):
#     return re.sub(r"http\S+", "", text)

# # Define the file name
# csv_file_name = "urdu_english.csv"

# # Combine English and Urdu sentences into rows
# cleaned_rows = []
# for english, urdu in zip(english_sentences, urdu_sentences):
#     cleaned_english = remove_urls(english)
#     cleaned_urdu = remove_urls(urdu)
#     cleaned_rows.append((cleaned_english, cleaned_urdu))

# # Write the cleaned rows to the CSV file
# with open(csv_file_name, mode='w', newline='', encoding='utf-8') as file:
#     writer = csv.writer(file)
#     writer.writerow(['english_sentence', 'urdu_sentence'])
#     writer.writerows(cleaned_rows)

# print("CSV file has been created successfully.")

In [8]:
english_vocabulary = [START_TOKEN, ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', 
                        '0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
                        ':', '<', '=', '>', '?', '@',
                        '[', '\\', ']', '^', '_', '`', '\'',
                        'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l',
                        'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 
                        'y', 'z', 
                        '{', '|', '}', '~', PADDING_TOKEN, END_TOKEN]

In [9]:
urdu_vocabulary = [START_TOKEN, ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', 
                        '0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
                        ':', '<', '=', '>', '?', '@',
                        '[', '\\', ']', '^', '_', '`', '\'',
                        'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l',
                        'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 
                        'y', 'z', 
                        '{', '|', '}', '~', PADDING_TOKEN, END_TOKEN]

In [10]:
index_to_urdu = {k:v for k,v in enumerate(urdu_vocabulary)}
urdu_to_index = {v:k for k,v in enumerate(urdu_vocabulary)}
index_to_english = {k:v for k,v in enumerate(english_vocabulary)}
english_to_index = {v:k for k,v in enumerate(english_vocabulary)}

In [11]:
import numpy as np
PERCENTILE = 97
print( f"{PERCENTILE}th percentile length urdu: {np.percentile([len(x) for x in urdu_sentences], PERCENTILE)}" )
print( f"{PERCENTILE}th percentile length English: {np.percentile([len(x) for x in english_sentences], PERCENTILE)}" )

97th percentile length urdu: 102.0
97th percentile length English: 94.0


In [12]:
max_sequence_length = 200

def is_valid_tokens(sentence, vocab):
    for token in list(set(sentence)):
        if token not in vocab:
            return False
    return True

def is_valid_length(sentence, max_sequence_length):
    return len(list(sentence)) < (max_sequence_length - 1) # need to re-add the end token so leaving 1 space

valid_sentence_indicies = []
for index in range(len(urdu_sentences)):
    urdu_sentence, english_sentence = urdu_sentences[index], english_sentences[index]
    if is_valid_length(urdu_sentence, max_sequence_length) \
      and is_valid_length(english_sentence, max_sequence_length) \
      and is_valid_tokens(urdu_sentence, urdu_vocabulary):
        valid_sentence_indicies.append(index)

print(f"Number of sentences: {len(urdu_sentences)}")
print(f"Number of valid sentences: {len(valid_sentence_indicies)}")

Number of sentences: 189102
Number of valid sentences: 186797


In [13]:
urdu_sentences = [urdu_sentences[i] for i in valid_sentence_indicies]
english_sentences = [english_sentences[i] for i in valid_sentence_indicies]

In [14]:
urdu_sentences[:3]

['film ka kya naam hai',
 'namaste, sada hua tomatoes score mahaan hai, lekin meta critic score is gunavatta kee philm se thoda kam lagata hai.',
 'kya aapako lagata hai ki aapako film pasand aaegee']

In [15]:
d_model = 512
batch_size = 1000
ffn_hidden = 2048
num_heads = 8
drop_prob = 0.1
num_layers = 1
max_sequence_length = 200
ur_vocab_size = len(urdu_vocabulary)

transformer = Transformer(d_model, 
                          ffn_hidden,
                          num_heads, 
                          drop_prob, 
                          num_layers, 
                          max_sequence_length,
                          ur_vocab_size,
                          english_to_index,
                          urdu_to_index,
                          START_TOKEN, 
                          END_TOKEN, 
                          PADDING_TOKEN)

In [16]:
transformer

Transformer(
  (encoder): Encoder(
    (sentence_embedding): SentenceEmbedding(
      (embedding): Embedding(69, 512)
      (position_encoder): PositionalEncoding()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (layers): SequentialEncoder(
      (0): EncoderLayer(
        (attention): MultiHeadAttention(
          (qkv_layer): Linear(in_features=512, out_features=1536, bias=True)
          (linear_layer): Linear(in_features=512, out_features=512, bias=True)
        )
        (norm1): LayerNormalization()
        (dropout1): Dropout(p=0.1, inplace=False)
        (ffn): PositionwiseFeedForward(
          (linear1): Linear(in_features=512, out_features=2048, bias=True)
          (linear2): Linear(in_features=2048, out_features=512, bias=True)
          (relu): ReLU()
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (norm2): LayerNormalization()
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (decoder): Decoder(
    (sentence_embedding):

In [17]:
from torch.utils.data import Dataset, DataLoader

class TextDataset(Dataset):

    def __init__(self, english_sentences, urdu_sentences):
        self.english_sentences = english_sentences
        self.urdu_sentences = urdu_sentences

    def __len__(self):
        return len(self.english_sentences)

    def __getitem__(self, idx):
        return self.english_sentences[idx], self.urdu_sentences[idx]
     

dataset = TextDataset(english_sentences, urdu_sentences)
     

len(dataset)

186797

In [18]:
dataset[1]

('hi, the rotten tomatoes score is great but the meta critic score seems a little low a movie of this quality. ',
 'namaste, sada hua tomatoes score mahaan hai, lekin meta critic score is gunavatta kee philm se thoda kam lagata hai.')

In [19]:
train_loader = DataLoader(dataset, batch_size)
iterator = iter(train_loader)
     

for batch_num, batch in enumerate(iterator):
    print(batch)
    if batch_num > 3:
        break

[("what's the name of the movie", 'hi, the rotten tomatoes score is great but the meta critic score seems a little low a movie of this quality. ', 'do you think you will like the movie', 'what kind of movie is it', 'when was the movie made?', 'wonder woman, i think i would enjoy this movie very much', 'whats the name of the movie', 'it is a action movie set in the dc comic world', 'who stars in the movie', 'the movie was made in 2015', 'did it do well in theaters?', 'did i have to see other movies to get it?', 'the main character is gal gadot, but also stars  chris pine, robin wright and danny huston', 'what is the movie about?', 'no i think this movie is a stand alone piece that you wont need knoledge of other movies to understand it', 'it is a war movie action movie', 'do you think this is a family movie for the kids', 'are you familiar with any of the actors in the movie?', 'some of them', 'i think its acceptable for a family to watch with children', 'even though its about war', 'wh

In [20]:
from torch import nn

criterian = nn.CrossEntropyLoss(ignore_index=urdu_to_index[PADDING_TOKEN],
                                reduction='none')

# When computing the loss, we are ignoring cases when the label is the padding token
for params in transformer.parameters():
    if params.dim() > 1:
        nn.init.xavier_uniform_(params)

optim = torch.optim.Adam(transformer.parameters(), lr=1e-4)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
     

NEG_INFTY = -1e9

def create_masks(eng_batch, ur_batch):
    num_sentences = len(eng_batch)
    look_ahead_mask = torch.full([max_sequence_length, max_sequence_length] , True)
    look_ahead_mask = torch.triu(look_ahead_mask, diagonal=1)
    encoder_padding_mask = torch.full([num_sentences, max_sequence_length, max_sequence_length] , False)
    decoder_padding_mask_self_attention = torch.full([num_sentences, max_sequence_length, max_sequence_length] , False)
    decoder_padding_mask_cross_attention = torch.full([num_sentences, max_sequence_length, max_sequence_length] , False)

    for idx in range(num_sentences):
      eng_sentence_length, ur_sentence_length = len(eng_batch[idx]), len(ur_batch[idx])
      eng_chars_to_padding_mask = np.arange(eng_sentence_length + 1, max_sequence_length)
      ur_chars_to_padding_mask = np.arange(ur_sentence_length + 1, max_sequence_length)
      encoder_padding_mask[idx, :, eng_chars_to_padding_mask] = True
      encoder_padding_mask[idx, eng_chars_to_padding_mask, :] = True
      decoder_padding_mask_self_attention[idx, :, ur_chars_to_padding_mask] = True
      decoder_padding_mask_self_attention[idx, ur_chars_to_padding_mask, :] = True
      decoder_padding_mask_cross_attention[idx, :, eng_chars_to_padding_mask] = True
      decoder_padding_mask_cross_attention[idx, ur_chars_to_padding_mask, :] = True

    encoder_self_attention_mask = torch.where(encoder_padding_mask, NEG_INFTY, 0)
    decoder_self_attention_mask =  torch.where(look_ahead_mask + decoder_padding_mask_self_attention, NEG_INFTY, 0)
    decoder_cross_attention_mask = torch.where(decoder_padding_mask_cross_attention, NEG_INFTY, 0)
    #print(encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask)
    return encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask

In [21]:
transformer.train()
transformer.to(device)
total_loss = 0
num_epochs = 10

for epoch in range(num_epochs):
    print(f"Epoch {epoch}")
    iterator = iter(train_loader)
    for batch_num, batch in enumerate(iterator):
        transformer.train()
        eng_batch, ur_batch = batch
        encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask = create_masks(eng_batch, ur_batch)
        optim.zero_grad()
        ur_predictions = transformer(eng_batch,
                                     ur_batch,
                                     encoder_self_attention_mask.to(device), 
                                     decoder_self_attention_mask.to(device), 
                                     decoder_cross_attention_mask.to(device),
                                     enc_start_token=False,
                                     enc_end_token=False,
                                     dec_start_token=True,
                                     dec_end_token=True)
        labels = transformer.decoder.sentence_embedding.batch_tokenize(ur_batch, start_token=False, end_token=True)
        loss = criterian(
            ur_predictions.view(-1, ur_vocab_size).to(device),
            labels.view(-1).to(device)
        ).to(device)
        valid_indicies = torch.where(labels.view(-1) == urdu_to_index[PADDING_TOKEN], False, True)
        loss = loss.sum() / valid_indicies.sum()
        loss.backward()
        optim.step()
        #train_losses.append(loss.item())
        if batch_num % 100 == 0:
            print(f"Iteration {batch_num} : {loss.item()}")
            print(f"English: {eng_batch[0]}")
            print(f"urdu Translation: {ur_batch[0]}")
            ur_sentence_predicted = torch.argmax(ur_predictions[0], axis=1)
            predicted_sentence = ""
            for idx in ur_sentence_predicted:
              if idx == urdu_to_index[END_TOKEN]:
                break
              predicted_sentence += index_to_urdu[idx.item()]
            print(f"urdu Prediction: {predicted_sentence}")


            transformer.eval()
            ur_sentence = ("",)
            eng_sentence = ("should we go to the mall?",)
            for word_counter in range(max_sequence_length):
                encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask= create_masks(eng_sentence, ur_sentence)
                predictions = transformer(eng_sentence,
                                          ur_sentence,
                                          encoder_self_attention_mask.to(device), 
                                          decoder_self_attention_mask.to(device), 
                                          decoder_cross_attention_mask.to(device),
                                          enc_start_token=False,
                                          enc_end_token=False,
                                          dec_start_token=True,
                                          dec_end_token=False)
                next_token_prob_distribution = predictions[0][word_counter] # not actual probs
                next_token_index = torch.argmax(next_token_prob_distribution).item()
                next_token = index_to_urdu[next_token_index]
                ur_sentence = (ur_sentence[0] + next_token, )
                if next_token == END_TOKEN:
                  break
            
            print(f"Evaluation translation (should we go to the mall?) : {ur_sentence}")
            print("-------------------------------------------")

Epoch 0
whats the name of the movie
hi, the rotten tomatoes score is great but the meta critic score seems a little low a movie of this quality. 
do you think you will like the movie
what kind of movie is it
when was the movie made?
wonder woman, i think i would enjoy this movie very much
whats the name of the movie
it is a action movie set in the dc comic world
who stars in the movie
the movie was made in 2015
did it do well in theaters?
did i have to see other movies to get it?
the main character is gal gadot, but also stars chris pine, robin wright and danny huston
what is the movie about?
no i think this movie is a stand alone piece that you wont need knoledge of other movies to understand it
it is a war movie action movie
do you think this is a family movie for the kids
are you familiar with any of the actors in the movie?
some of them
i think its acceptable for a family to watch with children
even though its about war
what kind of color pallet does it tend to use
yes they dont de

KeyError: '’'